In [183]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import random
import time 
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## 先只爬一篇文章的資訊、推文留言

In [184]:
payload={
'from':'/bbs/'+ 'Gossiping' +'/index.html',
'yes':'yes' 
}

url = 'https://www.ptt.cc/bbs/Examination/M.1582260742.A.8D9.html'
#url = 'https://www.ptt.cc/bbs/sex/M.1582985884.A.88A.html'
#url = 'https://www.ptt.cc/bbs/LoL/M.1583142390.A.3A0.html'
#破除18禁的限制
rs = requests.session()
req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
req = rs.get('https://www.ptt.cc/bbs/'+ 'sex' +'/index.html',verify = False)
req = rs.get(url, verify=False)

In [4]:
#200 means get success, 404 means fail
req.status_code

200

In [5]:
web_content = req.text
#以美味湯解析html
soup = BeautifulSoup(web_content, 'lxml')

In [6]:
main_content = soup.find('div',id="main-content")
metas = main_content.select('div.article-metaline')

In [7]:
author = metas[0].select('span.article-meta-value')[0].text
title = metas[1].select('span.article-meta-value')[0].text
post_date = metas[2].select('span.article-meta-value')[0].text

In [8]:
author_ID = author.split('(',1)[0].replace(' ','')
author_name = author.split('(',1)[1].replace(')','')

In [9]:
#content篩選出文章內文
content = soup.find(id="main-content").text
target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
content = content.split(target_content)
date = soup.select('.article-meta-value')[3].text
content = content[0].split(date)
content = content[1].replace('\n', '  ').replace('\t', '  ')

In [10]:
note = soup.select('span.f2')

In [11]:
org_url = note[1].text.replace('※ 文章網址: ','')

In [12]:
createdTime = dt.datetime.today().strftime("%d/%m/%Y %H:%M:%S")

In [13]:
createdTime

'03/03/2020 19:07:05'

In [14]:
post_info = []
post_info.append({'authorId':author_ID, 'authorName':author_name, 'title':title, 'publishedTime':post_date
                  , 'content':content, 'canonicalUrl':org_url, 'createdTime':createdTime})

In [15]:
post_info = pd.DataFrame(post_info)

In [16]:
pushes = main_content.find_all('div', class_='push')

In [19]:
messages = []
for push in pushes:
    push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
    push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
    # if find is None: find().strings -> list -> ' '.join; else the current way
    push_content = push.find('span', 'push-content').strings
    push_content = ' '.join(push_content)[1:].strip(' \t\n\r')  # remove ':'
    push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')    
    messages.append( {'canonicalUrl':org_url, 'push_tag': push_tag, 'commentId': push_userid
                      , 'commentContent': push_content, 'commentTime': push_ipdatetime} )


In [20]:
messages = pd.DataFrame(messages)

In [21]:
messages

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/Examination/M.158226074...,推,milk7054,第一年剛開始打底就別想太多，先當練筆,02/21 13:04
1,https://www.ptt.cc/bbs/Examination/M.158226074...,→,milk7054,6科專業，每科20章，一天看一科，這樣根本來不及,02/21 13:09
2,https://www.ptt.cc/bbs/Examination/M.158226074...,→,milk7054,光函授看完第一輪就差不多要考試,02/21 13:10
3,https://www.ptt.cc/bbs/Examination/M.158226074...,推,needeng,準時下班，怎會沒辦法看書？平日不行的話，你放假有看書,02/21 13:42
4,https://www.ptt.cc/bbs/Examination/M.158226074...,→,needeng,？你的企圖心不夠。,02/21 13:42
...,...,...,...,...,...
65,https://www.ptt.cc/bbs/Examination/M.158226074...,推,Curry16,人家全職考生窮到每天被家人念，你有薪水領，本來就要比,02/24 07:26
66,https://www.ptt.cc/bbs/Examination/M.158226074...,→,Curry16,較努力,02/24 07:26
67,https://www.ptt.cc/bbs/Examination/M.158226074...,推,h14753951,"過太爽就會沒動力念書,沒有壓力就沒有動力",02/24 12:23
68,https://www.ptt.cc/bbs/Examination/M.158226074...,推,h14753951,你應該好好找一下動力來源,02/24 12:27


In [22]:
pd.merge(post_info, messages, how='left', on='canonicalUrl')

,authorId,authorName,title,publishedTime,content,canonicalUrl,createdTime,push_tag,commentId,commentContent,commentTime
0,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,milk7054,第一年剛開始打底就別想太多，先當練筆,02/21 13:04
1,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,milk7054,6科專業，每科20章，一天看一科，這樣根本來不及,02/21 13:09
2,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,milk7054,光函授看完第一輪就差不多要考試,02/21 13:10
3,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,needeng,準時下班，怎會沒辦法看書？平日不行的話，你放假有看書,02/21 13:42
4,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,needeng,？你的企圖心不夠。,02/21 13:42
...,...,...,...,...,...,...,...,...,...,...,...
65,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,Curry16,人家全職考生窮到每天被家人念，你有薪水領，本來就要比,02/24 07:26
66,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,Curry16,較努力,02/24 07:26
67,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,h14753951,"過太爽就會沒動力念書,沒有壓力就沒有動力",02/24 12:23
68,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,h14753951,你應該好好找一下動力來源,02/24 12:27


## 抓某個看板的某個頁面的所有文章網址

In [28]:
payload={
'from':'/bbs/'+ 'sex' +'/index.html',
'yes':'yes' 
}

url = 'https://www.ptt.cc/bbs/Examination/index.html'
#破除18禁的限制
rs = requests.session()
req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
req = rs.get('https://www.ptt.cc/bbs/'+ 'sex' +'/index.html',verify = False)
req = rs.get(url, verify=False)

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

In [29]:
#200 means get success, 404 means fail
req.status_code

200

In [39]:
web_content = req.text
#以美味湯解析html
soup = BeautifulSoup(web_content, 'lxml')

In [48]:
post_list = soup.find_all('div',class_="title")

In [52]:
url_list = []
for item in post_list:
    try:
        url_list.append('https://www.ptt.cc'+item.select_one("a").get("href"))
    except: #刪文抓不到href
        pass

In [54]:
#得到該頁面的所有url
url_list

['https://www.ptt.cc/bbs/Examination/M.1582969553.A.FEE.html',
 'https://www.ptt.cc/bbs/Examination/M.1582979626.A.822.html',
 'https://www.ptt.cc/bbs/Examination/M.1583037348.A.AC9.html',
 'https://www.ptt.cc/bbs/Examination/M.1583038395.A.B8B.html',
 'https://www.ptt.cc/bbs/Examination/M.1583050146.A.269.html',
 'https://www.ptt.cc/bbs/Examination/M.1583050709.A.EB4.html',
 'https://www.ptt.cc/bbs/Examination/M.1583057947.A.E9E.html',
 'https://www.ptt.cc/bbs/Examination/M.1583069018.A.29E.html',
 'https://www.ptt.cc/bbs/Examination/M.1583075191.A.D9E.html',
 'https://www.ptt.cc/bbs/Examination/M.1583120282.A.3DB.html',
 'https://www.ptt.cc/bbs/Examination/M.1583133271.A.B3B.html',
 'https://www.ptt.cc/bbs/Examination/M.1583149220.A.F93.html',
 'https://www.ptt.cc/bbs/Examination/M.1583151717.A.0B7.html',
 'https://www.ptt.cc/bbs/Examination/M.1583160130.A.221.html',
 'https://www.ptt.cc/bbs/Examination/M.1583201813.A.888.html',
 'https://www.ptt.cc/bbs/Examination/M.1583216252.A.6F5

* 抓取該看板的總頁數

In [79]:
board = 'sex'
pre_num = soup.select('div.btn-group > a')[3]['href'].replace('/bbs/'+board+'/index','') ##
pre_num = pre_num.replace('.html','')
total_page_index = pd.to_numeric(pre_num)+1

ValueError: Unable to parse string "/bbs/Examination/index3007" at position 0

In [56]:
total_page_index

3008

In [57]:
for i in range(3008,3002,-1):
    print(i)

3008
3007
3006
3005
3004
3003


# 得到一篇文章資訊的function

In [226]:
def get_post_data(post_url):
    payload={
    'from':'/bbs/'+ 'Gossiping' +'/index.html',
    'yes':'yes' 
    }

    #破除18歲的限制
    rs = requests.session()
    req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
    req = rs.get('https://www.ptt.cc/bbs/'+ 'Gossiping' +'/index.html',verify = False)
    req = rs.get(post_url, verify=False)

    if req.status_code==200:
        web_content = req.text
        #以美味湯解析html
        soup = BeautifulSoup(web_content, 'lxml')
        main_content = soup.find('div',id="main-content")
        metas = main_content.select('div.article-metaline')

        author = metas[0].select('span.article-meta-value')[0].text
        title = metas[1].select('span.article-meta-value')[0].text
        post_date = metas[2].select('span.article-meta-value')[0].text
        author_ID = author.split('(',1)[0].replace(' ','')
        author_name = author.split('(',1)[1].replace(')','')

        #content篩選出文章內文
        content = soup.find(id="main-content").text
        target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        content = content.split(target_content)
        date = soup.select('.article-meta-value')[3].text
        content = content[0].split(date)
        content = content[1].replace('\n', '  ').replace('\t', '  ')

        #note = soup.select('span.f2')
        #org_url = note[1].text.replace('※ 文章網址: ','')
        createdTime = dt.datetime.today().strftime("%d/%m/%Y %H:%M:%S")
        post_info = []
        post_info.append({'authorId':author_ID, 'authorName':author_name, 'title':title, 'publishedTime':post_date
                          , 'content':content, 'canonicalUrl':post_url, 'createdTime':createdTime})
        post_info = pd.DataFrame(post_info)

        pushes = main_content.find_all('div', class_='push')
        messages = []
        for push in pushes:
            push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
            push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
            # if find is None: find().strings -> list -> ' '.join; else the current way
            push_content = push.find('span', 'push-content').strings
            push_content = ' '.join(push_content)[1:].strip(' \t\n\r')  # remove ':'
            push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')    
            messages.append( {'canonicalUrl':post_url, 'push_tag': push_tag, 'commentId': push_userid
                              , 'commentContent': push_content, 'commentTime': push_ipdatetime} )

        messages = pd.DataFrame(messages)

        #post_data = pd.merge(post_info, messages, how='left', on='canonicalUrl')

        return post_info, messages

    else:
        return "404"

In [134]:
get_post_data('https://www.ptt.cc/bbs/Gossiping/M.1583233903.A.A4C.html')[1]

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,→,WolfTeacher,嘔嘔嘔嘔,180.176.0.145 03/03 19:11
1,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,foolfighter,衣索比亞狗,223.137.205.227 03/03 19:12
2,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,kenek0612,五樓的洨,39.12.42.254 03/03 19:12
3,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,DiMammaMia,送他上西天,114.136.47.217 03/03 19:12
4,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,kaiyao,送他巧克力，祝他要到手指,111.241.103.23 03/03 19:12
5,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,→,will0620,祖國的善液,223.136.24.151 03/03 19:12
6,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,→,Yude0109,R.I.P,59.115.68.236 03/03 19:12
7,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,railwaytaiwa,祝他像老柯一樣拉肚……,36.237.211.250 03/03 19:12
8,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,patiger,請他吃拔辣,114.39.84.185 03/03 19:12
9,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,hqu,阿兩也是這一天生日,42.77.65.70 03/03 19:13


# 再寫一個可以看要爬幾頁的function，產出href_list

In [232]:
def get_href_from_page(board_name, scrap_page):
    payload={
    'from':'/bbs/'+ board_name +'/index.html',
    'yes':'yes' 
    }

    #破除18歲的限制
    rs = requests.session()
    req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
    req = rs.get('https://www.ptt.cc/bbs/'+ board_name +'/index.html',verify = False)
    if req.status_code==200:
        web_content = req.text
        #以美味湯解析html
        soup = BeautifulSoup(web_content, 'lxml')
        pre_num = soup.select('div.btn-group > a')[3]['href'].replace('/bbs/'+board_name+'/index','') ##
        pre_num = pre_num.replace('.html','')
        total_page_index = pd.to_numeric(pre_num)+1

        url_list = []
        for i in range(total_page_index, total_page_index-scrap_page, -1):
            req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
            req = rs.get('https://www.ptt.cc/bbs/'+ board_name +'/index'+str(i)+'.html',verify = False)
            web_content = req.text
            #以美味湯解析html
            soup = BeautifulSoup(web_content, 'lxml')
            post_list = soup.find_all('div',class_="title")
            for item in post_list:
                try:
                    url_list.append('https://www.ptt.cc'+item.select_one("a").get("href"))
                except: #刪文抓不到href
                    pass         
        return url_list
    
    else:
        return "404"

# 開始寫 main

In [236]:
def main(Board_Name, Scrap_Page):
    post_list = get_href_from_page(board_name=Board_Name, scrap_page=Scrap_Page)
    all_post_info = get_post_data(post_list[0])[0] 
    for i in range(1, len(post_list)):
        try:
            all_post_info = all_post_info.append(get_post_data(post_list[i])[0])
        except:
            pass

    all_msg = get_post_data(post_list[0])[1] 
    for i in range(1, len(post_list)):
        try:
            all_msg = all_msg.append(get_post_data(post_list[i])[1])    
        except:
            pass

        
    return all_post_info, all_msg

In [238]:
main('AGEC88',5)[0] 

,authorId,authorName,title,publishedTime,content,canonicalUrl,createdTime
0,BABYMING,you're my treasure,班聚？？,Thu Jan 19 08:52:24 2012,今年要不要辦班聚阿？ 感覺上次聚會又過兩年了 -- ※ 發信站: 批踢...,https://www.ptt.cc/bbs/AGEC88/M.1326934346.A.5...,03/03/2020 22:13:47
0,dfj,。與主同行。,Re: 班聚？？,Fri Jan 20 13:50:39 2012,※ 引述《BABYMING (you're my treasure)》之銘言： : 今...,https://www.ptt.cc/bbs/AGEC88/M.1327038642.A.3...,03/03/2020 22:13:49
0,dfj,。與主同行。,[閒聊] 以前的我們好青春喔！,Sun Sep 30 15:59:06 2012,下午為了記起一個遺忘的名字， 看了一下班版的精華區， 發現以前的我們真的...,https://www.ptt.cc/bbs/AGEC88/M.1348991955.A.E...,03/03/2020 22:13:51
0,avstar,天涼好個秋,通訊錄更新,Tue Dec 25 23:02:54 2012,"還有人在看這個版嗎??? 大家快去收信 更新一下通訊錄吧~ 目前只有""...",https://www.ptt.cc/bbs/AGEC88/M.1356447777.A.5...,03/03/2020 22:13:52
0,avstar,天涼好個秋,新年快樂,Tue Jan 1 20:56:03 2013,呵呵 又過一年了 -- ※ 發信站: 批踢踢實業坊(ptt.cc) ◆ F...,https://www.ptt.cc/bbs/AGEC88/M.1357044966.A.6...,03/03/2020 22:13:54
...,...,...,...,...,...,...,...
0,pamelah,相信,Re: [公告] 各位同學 辦同學會啦,Fri Jun 2 15:36:07 2006,※ 引述《BABYMING (you're my treasure)》之銘言： : 離...,https://www.ptt.cc/bbs/AGEC88/M.1149233593.A.A...,03/03/2020 22:17:28
0,dfj,愛，應該沒有怨言,Re: [公告] 各位同學 辦同學會啦,Fri Jun 2 16:55:38 2006,※ 引述《pamelah (相信)》之銘言： : ※ 引述《BABYMING (you...,https://www.ptt.cc/bbs/AGEC88/M.1149238412.A.8...,03/03/2020 22:17:33
0,Eloy,hope,Re: [公告] 各位同學 辦同學會啦,Sat Jun 3 10:40:42 2006,※ 引述《dfj (愛，應該沒有怨言 )》之銘言： : ※ 引述《pamelah (相...,https://www.ptt.cc/bbs/AGEC88/M.1149301733.A.E...,03/03/2020 22:17:33
0,avstar,天涼好個秋,Re: [公告] 各位同學 辦同學會啦,Sun Jun 4 08:36:04 2006,※ 引述《BABYMING (you're my treasure)》之銘言： : 離...,https://www.ptt.cc/bbs/AGEC88/M.1149381347.A.D...,03/03/2020 22:17:35


In [239]:
main('AGEC88',5)[1] 

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/AGEC88/M.1348991955.A.E...,推,BABYMING,我也很高興跟大家當同學^^,10/01 21:46
0,https://www.ptt.cc/bbs/AGEC88/M.1356447777.A.5...,推,dfj,好期待「十週年畢業感恩同學會」喔！,12/27 16:47
0,https://www.ptt.cc/bbs/AGEC88/M.1376066385.A.F...,推,dfj,同學會呢？,08/30 13:03
0,https://www.ptt.cc/bbs/AGEC88/M.1392388175.A.4...,推,dfj,祝大家天天快樂！,02/21 12:44
0,https://www.ptt.cc/bbs/AGEC88/M.1393975370.A.9...,推,dfj,辛苦了!,04/01 16:13
...,...,...,...,...,...
5,https://www.ptt.cc/bbs/AGEC88/M.1169655425.A.7...,→,indy,有空過年前湊一團來個聚餐吧!!,02/08 23:04
6,https://www.ptt.cc/bbs/AGEC88/M.1169655425.A.7...,推,dfj,轉眼間，好快就要過年了耶..好像也沒什麼消息..,02/11 01:11
0,https://www.ptt.cc/bbs/AGEC88/M.1171723570.A.B...,推,dfj,May God Bless All of you!!,02/26 11:21
1,https://www.ptt.cc/bbs/AGEC88/M.1171723570.A.B...,推,chiwhe921,艀w,10/10 23:32


In [237]:
main('sex',2)[0]

,authorId,authorName,title,publishedTime,content,canonicalUrl,createdTime
0,Assisi,Francesco d'Assisi,[公告] 2020 叁月 西斯置底閒聊區,Tue Mar 3 12:01:59 2020,https://i.imgur.com/rlxvpFA.jpg 嗯... 巷子...,https://www.ptt.cc/bbs/sex/M.1583208123.A.F59....,03/03/2020 22:05:44
0,nini21,小魯宅,Re: [閒聊] 高瘦的男生是不是,Tue Mar 3 12:42:38 2020,※ 引述《pzhao (睡覺青年)》之銘言： : 根據小弟與某些女伴的討論 : 以及...,https://www.ptt.cc/bbs/sex/M.1583210560.A.335....,03/03/2020 22:05:47
0,goldenrose,蓋世英雄,Re: [討論] 女生願意被幹嘴巴的比例高嗎,Tue Mar 3 15:40:27 2020,因為本身是口交控(含到底超讚) 看到這篇實在忍不住想回 然後我是男生 不要看第一句...,https://www.ptt.cc/bbs/sex/M.1583221229.A.059....,03/03/2020 22:05:48
0,zkow,逍遙山水憶秋年,[新聞] 高中女生讓他「凍沒條」色師廁所裝針孔,Tue Mar 3 16:24:02 2020,高中女生讓他「凍沒條」 色師廁所裝針孔辯：才裝一小時什麼都沒拍到 記者沈繼昌／...,https://www.ptt.cc/bbs/sex/M.1583223844.A.DE0....,03/03/2020 22:05:48
0,pzhao,睡覺青年,[閒聊] 有喜歡吞精的女生嗎,Tue Mar 3 16:33:03 2020,在吃茶葉蛋時想到 喜歡吃茶葉蛋的人很多 但愛吃蛋黃的人卻是少之又少 如...,https://www.ptt.cc/bbs/sex/M.1583224385.A.C2A....,03/03/2020 22:05:51
0,abingla0705,啡啡,[閒聊] 玩具。,Tue Mar 3 18:44:46 2020,與你賣力進出我的下身不同 拍落在臉頰上的手 感覺得出來力道是控制過的...,https://www.ptt.cc/bbs/sex/M.1583232288.A.196....,03/03/2020 22:05:53
0,CrystalNik,水晶尼克,[閒聊] 抱睡真D爽,Tue Mar 3 19:49:33 2020,抱睡真滴爽 有時候你就真的只是想有一個人純抱抱 陪你睡覺 尤其是...,https://www.ptt.cc/bbs/sex/M.1583236175.A.0C2....,03/03/2020 22:05:54
0,drty,no name,[閒聊] 按摩棒的使用方法,Tue Mar 3 20:17:35 2020,前陣子趁著情人節特價 買了一隻按摩棒 不過用了幾次不是覺得進入的時候不舒服 就...,https://www.ptt.cc/bbs/sex/M.1583237857.A.334....,03/03/2020 22:05:56
0,orz44444,陰人拜陰廟=小鬼拜大鬼~~,[新聞] 基隆艦遭外流艦上男女士官兵性愛片 竟是男上士偷錄,Tue Mar 3 20:18:03 2020,https://udn.com/news/story/7317/4385883 2...,https://www.ptt.cc/bbs/sex/M.1583237890.A.C0F....,03/03/2020 22:05:57
0,falcon1014,忠,[心得] 2011-花美男聚會,Tue Mar 3 20:23:14 2020,整理加密硬碟，看著影像和文章，滿滿的回憶， 本文場景在2011年，還是PC上網...,https://www.ptt.cc/bbs/sex/M.1583238209.A.6CA....,03/03/2020 22:05:58


In [227]:
get_post_data('https://www.ptt.cc/bbs/sex/M.1583221229.A.059.html')[0] 

,authorId,authorName,title,publishedTime,content,canonicalUrl,createdTime
0,goldenrose,蓋世英雄,Re: [討論] 女生願意被幹嘴巴的比例高嗎,Tue Mar 3 15:40:27 2020,因為本身是口交控(含到底超讚) 看到這篇實在忍不住想回 然後我是男生 不要看第一句...,https://www.ptt.cc/bbs/sex/M.1583221229.A.059....,03/03/2020 21:47:25


In [139]:
AGEC_list = get_href_from_page(board_name='AGECservice', scrap_page=1)

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

In [158]:
AGEC_list[0]

'https://www.ptt.cc/bbs/AGECservice/M.1369038539.A.72F.html'

In [174]:
all_post_info = get_post_data(AGEC_list[0])[0] 
for i in range(1, len(AGEC_list)):
    all_post_info = all_post_info.append(get_post_data(AGEC_list[i])[0])



/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

In [185]:
all_msg = get_post_data(AGEC_list[0])[1] 
for i in range(1, len(AGEC_list)):
    all_msg = all_msg.append(get_post_data(AGEC_list[i])[1])

In [186]:
all_msg

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/AGECservice/M.137137434...,推,yf87008,cool,06/18 01:26
0,https://www.ptt.cc/bbs/AGECservice/M.137205577...,推,etwu0718,,06/24 20:29
0,https://www.ptt.cc/bbs/AGECservice/M.133104532...,推,achun8184,夾夾讚 頭推,03/06 23:57
1,https://www.ptt.cc/bbs/AGECservice/M.133104532...,→,onemoreshot,快點開工開始做事比較實在！有問題快問,03/07 01:11
2,https://www.ptt.cc/bbs/AGECservice/M.133104532...,推,pgking1224,何世堯大教案好像是管理組...,03/15 22:07
0,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,bloodyjimi,3/26 農藝組 10:00~12:00 13:00~17:00 tks!,03/16 14:58
1,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,hauerhsieh,3/22 全天共六小時 B97607019 農經四 謝政豪 謝夾,03/20 14:10
2,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,syric,4/4 農藝組 1000-1200 1300-1500 農經二 黃崇榮 謝謝！,03/25 20:10
3,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,syric,不好意思 想提前一天 改成4/3(二) 謝謝！,03/28 07:23
4,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,syric,上次登記春假但農場沒開 想再登記4/20(五) 1300-1700 謝謝!,04/13 13:21
